In [10]:
import numpy as np
import pandas as pd


In [11]:
dt_guru = "../dataset/dg.xlsx"
dt_sekolah = "../dataset/ds.xlsx"

df_guru = pd.read_excel(dt_guru)
df_sekolah = pd.read_excel(dt_sekolah)

In [12]:
df_guru.head(), df_sekolah.head()

(   Id Guru            Nama Guru Jenis Kelamin      LatG       LongG
 0        3       Siti Juwariyah             P -7.608048  110.243011
 1       15         ENI ZARYANTI             P -7.557934  110.244500
 2       21     Wahyu Rakhmawati             P -7.745043  109.396167
 3       22  Purwoko Budisantoso             L -7.824374  110.262371
 4       30            Sabariyah             P -7.417245  110.184932,
    id                     nama  rombel    lats     longs
 0  45    SD NEGERI NGARGOGONDO       6 -7.6263  110.2094
 1  48  SD NEGERI RINGINPUTIH 3       6 -7.5994  110.1963
 2  51    SD NEGERI TANJUNGSARI       6 -7.6171  110.1974
 3  56       SD NEGERI BENINGAN       6 -7.5154  110.2900
 4  67       SD NEGERI PODOSOKO       6 -7.5183  110.2568)

In [13]:
# Function to calculate the Haversine distance
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in kilometers
    dLat = np.radians(lat2 - lat1)
    dLon = np.radians(lon2 - lon1)
    a = np.sin(dLat / 2) ** 2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dLon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

In [22]:
# Get the coordinates of gurus and schools
coords_guru = df_guru[['LatG', 'LongG']].to_numpy()
coords_sekolah = df_sekolah[['lats', 'longs']].to_numpy()

# Calculate the distance matrix
distance_matrix = np.zeros((len(coords_guru), len(coords_sekolah)))

for i, (lat_g, lon_g) in enumerate(coords_guru):
    for j, (lat_s, lon_s) in enumerate(coords_sekolah):
        distance_matrix[i, j] = haversine(lat_g, lon_g, lat_s, lon_s)
        
distance_matrix

array([[ 4.223913  ,  5.23737959,  5.12677839, ..., 25.89712243,
        24.49444505, 25.4298384 ],
       [ 8.52975574,  7.03457867,  8.3805535 , ..., 21.27132056,
        19.45332655, 20.65049773],
       [90.58253111, 89.64917021, 89.43247584, ..., 89.4036346 ,
        92.90793949, 90.64365558],
       ...,
       [26.58607739, 26.29232587, 27.14557325, ..., 32.14150471,
        28.63547684, 30.94990047],
       [ 4.9060296 ,  1.5868795 ,  3.51277553, ..., 21.36518663,
        20.69286765, 21.13604092],
       [20.45036179, 17.69272763, 18.7843947 , ..., 16.01556759,
        18.4278636 , 16.85588508]])

In [15]:
class AntColonyAssignment:
    def __init__(self, distances, n_ants, n_iterations, decay, alpha=1, beta=2):
        self.distances = distances
        self.pheromone = np.ones(self.distances.shape) / len(distances)
        self.all_gurus = range(distances.shape[0])
        self.all_schools = range(distances.shape[1])
        self.n_ants = n_ants
        self.n_iterations = n_iterations
        self.decay = decay
        self.alpha = alpha
        self.beta = beta

    def run(self):
        shortest_path = None
        best_distance = np.inf
        for _ in range(self.n_iterations):
            all_paths = self.gen_all_paths()
            self.spread_pheromone(all_paths)
            min_path = min(all_paths, key=lambda x: x[1])
            if min_path[1] < best_distance:
                shortest_path = min_path
                best_distance = min_path[1]
            self.pheromone *= self.decay
        return shortest_path

    def spread_pheromone(self, all_paths):
        sorted_paths = sorted(all_paths, key=lambda x: x[1])
        for path, dist in sorted_paths[:self.n_ants]:
            for guru, school in path:
                self.pheromone[guru, school] += 1.0 / dist

    def gen_path_dist(self, path):
        total_dist = 0
        for guru, school in path:
            total_dist += self.distances[guru, school]
        return total_dist

    def gen_all_paths(self):
        all_paths = []
        for _ in range(self.n_ants):
            path = self.gen_path()
            all_paths.append((path, self.gen_path_dist(path)))
        return all_paths

    def gen_path(self):
        path = []
        available_schools = list(self.all_schools)
        for guru in self.all_gurus:
            if available_schools:
                move = self.pick_move(guru, available_schools)
                path.append((guru, move))
                available_schools.remove(move)
            else:
                break
        return path

    def pick_move(self, guru, available_schools):
        pheromone = self.pheromone[guru, available_schools]
        distances = self.distances[guru, available_schools]
        row = pheromone ** self.alpha * ((1.0 / distances) ** self.beta)
        norm_row = row / row.sum()
        move = np.random.choice(available_schools, p=norm_row)
        return move


In [16]:
# Inisialisasi parameter ACO
n_ants = 10
n_iterations = 500
decay = 0.95
alpha = 1
beta = 2

ant_colony = AntColonyAssignment(distance_matrix, n_ants, n_iterations, decay, alpha, beta)
shortest_path = ant_colony.run()

print(f"Penempatan guru di sekolah dengan jarak total minimal: {shortest_path[0]}")
print(f"Jarak total: {shortest_path[1]}")

Penempatan guru di sekolah dengan jarak total minimal: [(0, 35), (1, 33), (2, 21), (3, 47), (4, 104), (5, 16), (6, 45), (7, 51), (8, 87), (9, 68), (10, 48), (11, 19), (12, 52), (13, 0), (14, 2), (15, 50), (16, 24), (17, 53), (18, 66), (19, 49), (20, 73), (21, 37), (22, 36), (23, 30), (24, 28), (25, 84), (26, 105), (27, 89), (28, 99), (29, 5), (30, 82), (31, 79), (32, 83), (33, 80), (34, 77), (35, 6), (36, 67), (37, 65), (38, 90), (39, 88), (40, 1), (41, 91), (42, 94), (43, 63), (44, 58), (45, 95), (46, 10), (47, 23), (48, 14), (49, 27), (50, 46), (51, 32), (52, 93), (53, 8), (54, 4), (55, 76), (56, 78), (57, 31), (58, 72), (59, 69), (60, 70), (61, 74), (62, 71), (63, 15), (64, 3), (65, 96), (66, 92), (67, 64), (68, 59), (69, 54), (70, 61), (71, 56), (72, 102), (73, 18), (74, 100), (75, 97), (76, 101), (77, 25), (78, 85), (79, 9), (80, 57), (81, 62), (82, 60), (83, 103), (84, 42), (85, 38), (86, 13), (87, 86), (88, 41), (89, 40), (90, 44), (91, 12), (92, 7), (93, 81), (94, 39), (95, 55)